AGENTE DE IA PARA ANÁLISE EXPLORATÓRIA DE DADOS

Bem-vindo ao **AGENTE EDA INTELIGENTE**!

**O que este agente pode fazer:**
- **Análise de CSV:** Faça upload de qualquer arquivo CSV para iniciar a análise.
- **Processamento de Linguagem Natural:** Faça perguntas em português e receba insights detalhados gerados pela API Google Gemini.
- **Análises Automáticas:** Execute análises de tipos de dados, estatísticas, correlações e detecção de outliers com um clique.
- **Geração de Gráficos:** Crie e visualize gráficos interativos, que também são salvos como imagens.
- **Memória e Conclusões:** O agente mantém um histórico das análises para gerar conclusões abrangentes e permitir o download dos resultados.

**▶️ Como Usar:**
1.  **Configure a API Key:** Siga as instruções na Célula 2.
2.  **Execute as Células em Ordem:** A melhor forma é ir ao menu `Ambiente de execução` e selecionar `Executar tudo`.
3.  **Acesse a Interface:** Após a execução, clique no link público do Gradio que será gerado no final para abrir o agente.

### **1) INSTALAÇÃO DE DEPENDÊNCIAS**

In [ ]:
# CÉLULA 1: INSTALAÇÃO UNIFICADA DE DEPENDÊNCIAS
!pip install -q google-generativeai gradio pandas numpy matplotlib seaborn plotly scikit-learn openpyxl
!pip install -q -U kaleido
print("✅ Todas as dependências foram instaladas com sucesso.")

✅ Todas as dependências foram instaladas com sucesso.


### **2) CONFIGURAÇÃO DA CHAVE DA API GEMINI**

In [ ]:
# CÉLULA 2: VERIFICAÇÃO DA API KEY NOS SECRETS DO COLAB
from google.colab import userdata

try:
    api_key = userdata.get('GEMINI_API_KEY')
    if api_key and len(api_key) > 20:
        print("✅ API Key configurada corretamente!")
        print(f"🔑 Chave encontrada (início): {api_key[:10]}...")
        print("🚀 Pronto para prosseguir!")
    else:
        print("⚠️ API Key muito curta ou inválida.")
        print("🔧 Verifique se copiou a chave completa.")
        raise ValueError("API Key inválida.")
except Exception as e:
    print("❌ API Key não encontrada!")
    print("🔧 Configure GEMINI_API_KEY nos secrets do Colab:")
    print("   1. Clique no ícone 🔑 na barra lateral esquerda.")
    print("   2. Crie um 'new secret' com o nome: GEMINI_API_KEY")
    print("   3. Cole sua chave obtida em: https://makersuite.google.com/app/apikey")
    print("   4. Ative a opção 'Acesso ao notebook'.")
    raise ValueError("Configure a API Key antes de continuar!")

✅ API Key configurada corretamente!
🔑 Chave encontrada (início): AIzaSyAHPF...
🚀 Pronto para prosseguir!


### **3) CÓDIGO PRINCIPAL DO AGENTE EDA**

In [ ]:
# CÉLULA 3: DEFINIÇÃO DO AGENTE E DA INTERFACE (VERSÃO UNIFICADA E MELHORADA)

# Instalação isolada de kaleido dentro desta célula para garantir acesso ao ambiente
!pip install -q -U kaleido
print("✅ Pacote kaleido instalado/atualizado dentro da CÉLULA 3.")


import os
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import uuid
# import plotly.io as pio # Não precisamos mais importar pio explicitamente para to_image()
import base64 # Importando base64 para possível uso futuro

warnings.filterwarnings('ignore')

import google.generativeai as genai
from google.colab import userdata
import gradio as gr
from typing import Dict, List, Any, Tuple, Optional

# Importações para geração de PDF
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image

# Diretório para salvar gráficos temporariamente (ainda útil para outras coisas se necessário)
temp_dir = "temp_plots"
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)

class CSVAnalysisAgent:
    """Agente de IA especializado em Análise Exploratória de Dados (EDA) para arquivos CSV"""

    def __init__(self, api_key: str):
        genai.configure(api_key=api_key)
        self.model = genai.GenerativeModel('gemini-2.5-flash')
        self.memory: List[Dict[str, Any]] = []
        self.current_data: Optional[pd.DataFrame] = None
        self.data_summary: Optional[Dict[str, Any]] = None
        self.generated_image_bytes: Dict[str, bytes] = {} # Armazenar bytes das imagens geradas (para PDF, se funcionar)
        self.generated_figures: Dict[str, go.Figure] = {} # Armazenar objetos Figure gerados


    def _clear_temp_plots(self):
        """Limpa o diretório de plots temporários para evitar acúmulo de arquivos."""
        if os.path.exists(temp_dir):
            for filename in os.listdir(temp_dir):
                file_path = os.path.join(temp_dir, filename)
                try:
                    if os.path.isfile(file_path):
                        os.unlink(file_path)
                except Exception as e:
                    print(f"Falha ao deletar {file_path}. Motivo: {e}")
        self.generated_image_bytes.clear() # Limpar bytes de imagem também
        self.generated_figures.clear() # Limpar figuras também


    def load_csv_file(self, file_path: str) -> str:
        """Carrega um arquivo CSV para análise e limpa análises anteriores."""
        self._clear_temp_plots() # MELHORIA: Limpa gráficos antigos ao carregar novo arquivo
        self.memory = [] # Reseta a memória
        try:
            self.current_data = pd.read_csv(file_path)
            self.data_summary = self._generate_data_summary()
            memory_entry = {
                "action": "load_data",
                "timestamp": pd.Timestamp.now(),
                "data_shape": self.current_data.shape,
                "columns": list(self.current_data.columns)
            }
            self.memory.append(memory_entry)
            return f"Dados carregados com sucesso! Dimensão: {self.current_data.shape}"
        except Exception as e:
            self.current_data = None
            self.data_summary = None
            return f"Erro ao carregar arquivo: {str(e)}"

    def _generate_data_summary(self) -> Optional[Dict[str, Any]]:
        if self.current_data is None: return None
        return {
            "shape": self.current_data.shape,
            "columns": list(self.current_data.columns),
            "dtypes": self.current_data.dtypes.astype(str).to_dict(),
            "missing_values": self.current_data.isnull().sum().to_dict(),
            "numeric_columns": list(self.current_data.select_dtypes(include=[np.number]).columns),
            "categorical_columns": list(self.current_data.select_dtypes(include=['object']).columns)
        }

    def analyze_data_types(self) -> Tuple[str, Optional[go.Figure]]:
        if self.current_data is None or self.data_summary is None: return "Nenhum dado carregado.", None
        analysis = {
            "Numéricos": len(self.data_summary["numeric_columns"]),
            "Categóricos": len(self.data_summary["categorical_columns"]),
            "Total de Colunas": len(self.data_summary["columns"])
        }
        fig = px.bar(x=list(analysis.keys()), y=list(analysis.values()), title="Distribuição dos Tipos de Dados")
        self.generated_figures['data_types_plot'] = fig # Armazenar figura
        img_bytes = None
        try:
            img_bytes = fig.to_image(format="png") # Gerar bytes da imagem (tentativa para PDF)
            self.generated_image_bytes['data_types_plot'] = img_bytes # Armazenar bytes
            print("✅ Bytes da imagem 'data_types_plot' gerados e armazenados (se kaleido funcionar).")
        except Exception as e:
            print(f"⚠️ Erro ao gerar bytes da imagem 'data_types_plot' (kaleido): {e}")

        self.memory.append({"action": "analyze_data_types", "timestamp": pd.Timestamp.now(), "analysis": analysis, "image_key": 'data_types_plot' if img_bytes else None})
        return str(analysis), fig # Retornar apenas texto e figura


    def statistical_summary(self) -> str:
        if self.current_data is None or self.data_summary is None: return "Nenhum dado carregado."
        numeric_stats = self.current_data.describe().to_string()
        categorical_stats = {col: {"valores_unicos": int(self.current_data[col].nunique()), "mais_frequente": str(self.current_data[col].mode().iloc[0])} for col in self.data_summary["categorical_columns"]}
        summary_text = f"Estatísticas Numéricas:\n{numeric_stats}\n\nEstatísticas Categóricas:\n{categorical_stats}"
        self.memory.append({"action": "statistical_summary", "timestamp": pd.Timestamp.now(), "summary": summary_text})
        return summary_text

    def detect_outliers(self, column_name: Optional[str] = None) -> Tuple[str, Optional[go.Figure]]:
        if self.current_data is None or self.data_summary is None: return "Nenhum dado carregado.", None
        numeric_cols = self.data_summary["numeric_columns"]
        cols_to_analyze = [column_name] if (column_name and column_name in numeric_cols) else numeric_cols[:4]
        if not cols_to_analyze: return "Não há colunas numéricas para analisar.", None
        fig = make_subplots(rows=1, cols=len(cols_to_analyze), subplot_titles=cols_to_analyze)
        outliers_info = {}
        for i, col in enumerate(cols_to_analyze):
            series = self.current_data[col].dropna()
            if not series.empty:
                Q1, Q3 = series.quantile(0.25), series.quantile(0.75)
                IQR = Q3 - Q1
                lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
                outliers = self.current_data[(self.current_data[col] < lower) | (self.current_data[col] > upper)]
                outliers_info[col] = {"contagem": len(outliers), "percentual": (len(outliers) / len(self.current_data)) * 100}
                fig.add_trace(go.Box(y=self.current_data[col], name=col), row=1, col=i + 1)
        fig.update_layout(title="Detecção de Outliers - Boxplots")
        info_text = f"Informação de Outliers: {outliers_info}"
        self.generated_figures['outliers_plot'] = fig # Armazenar figura
        img_bytes = None
        try:
            img_bytes = fig.to_image(format="png") # Gerar bytes da imagem (tentativa para PDF)
            self.generated_image_bytes['outliers_plot'] = img_bytes # Armazenar bytes
            print("✅ Bytes da imagem 'outliers_plot' gerados e armazenados (se kaleido funcionar).")
        except Exception as e:
            print(f"⚠️ Erro ao gerar bytes da imagem 'outliers_plot' (kaleido): {e}")


        self.memory.append({"action": "detect_outliers", "timestamp": pd.Timestamp.now(), "outliers_info": outliers_info, "image_key": 'outliers_plot' if img_bytes else None})
        return info_text, fig # Retornar apenas texto e figura


    def correlation_analysis(self) -> Tuple[str, Optional[go.Figure]]:
        if self.current_data is None: return "Nenhum dado carregado.", None
        numeric_data = self.current_data.select_dtypes(include=[np.number])
        if numeric_data.shape[1] < 2: return "Poucas colunas numéricas para correlação.", None
        corr_matrix = numeric_data.corr(numeric_only=True)
        fig = px.imshow(corr_matrix, title="Matriz de Correlação", color_continuous_scale="RdBu", aspect="auto")
        high_corr = {f"{c1}-{c2}": corr_matrix.loc[c1, c2] for c1 in corr_matrix.columns for c2 in corr_matrix.columns if c1 != c2 and abs(corr_matrix.loc[c1, c2]) > 0.7}
        info_text = f"Correlações Altas (abs > 0.7): {high_corr}"
        self.generated_figures['correlation_plot'] = fig # Armazenar figura
        img_bytes = None
        try:
            img_bytes = fig.to_image(format="png") # Gerar bytes da imagem (tentativa para PDF)
            self.generated_image_bytes['correlation_plot'] = img_bytes # Armazenar bytes
            print("✅ Bytes da imagem 'correlation_plot' gerados e armazenados (se kaleido funcionar).")
        except Exception as e:
            print(f"⚠️ Erro ao gerar bytes da imagem 'correlation_plot' (kaleido): {e}")

        self.memory.append({"action": "correlation_analysis", "timestamp": pd.Timestamp.now(), "high_correlations": high_corr, "image_key": 'correlation_plot' if img_bytes else None})
        return info_text, fig # Retornar apenas texto e figura

    def save_plots_as_html(self) -> List[str]:
        """Salva os gráficos gerados como arquivos HTML no diretório temporário."""
        saved_files = []
        print(f"ℹ️ Tentando salvar {len(self.generated_figures)} figura(s) como HTML individual...") # Depuração
        if not self.generated_figures:
            print("⚠️ Nenhuma figura gerada para salvar como HTML.")
            return []

        for key, fig in self.generated_figures.items():
            html_filename = os.path.join(temp_dir, f"{key}.html")
            try:
                fig.write_html(html_filename)
                saved_files.append(html_filename)
                print(f"✅ Figura '{key}' salva como HTML em {html_filename}") # Depuração
            except Exception as e:
                print(f"❌ Erro ao salvar figura '{key}' como HTML: {e}") # Depuração

        print(f"✅ Concluído o salvamento de gráficos individuais. Total de arquivos salvos: {len(saved_files)}") # Depuração
        return saved_files

    def _format_memory(self) -> str:
        if not self.memory: return "Nenhuma análise anterior."
        formatted = []
        for entry in self.memory[-10:]:
            action = entry.get("action", "").replace('_', ' ').title()
            # Referenciar a chave da imagem se existir
            img_info = f" | Imagem Key: {entry['image_key']}" if entry.get('image_key') else ""
            if action == "Query":
                response_snippet = entry['response'][:150] + '...' if len(entry['response']) > 150 else entry['response']
                formatted.append(f"- Pergunta: {entry['question']} -> Resposta: {response_snippet}")
            else:
                formatted.append(f"- Análise: {action}{img_info}")
        return "\n".join(formatted)

    def get_conclusions(self) -> str:
        if self.current_data is None or self.data_summary is None: return "Nenhum dado carregado."
        context = f"""Baseado em todas as análises realizadas no dataset (histórico na memória), gere conclusões abrangentes. Inclua referências aos nomes dos arquivos de imagem dos gráficos gerados, quando relevante (use os 'Image Key's do histórico se disponíveis).
Histórico de Análises:\n{self._format_memory()}\n\nForneça em português, com tópicos claros:\n1. Principais insights descobertos.\n2. Padrões e tendências identificados.\n3. Avaliação da qualidade dos dados (valores faltantes, outliers).\n4. Recomendações para análises futuras ou ações de negócio."""
        try:
            response = self.model.generate_content(context)
            return response.text if hasattr(response, 'text') and response.text else "O modelo não gerou conclusões."
        except Exception as e:
            return f"Erro ao gerar conclusões: {str(e)}"

    def generate_pdf_report(self) -> Optional[str]:
        """Gera um relatório PDF com conclusões e gráficos."""
        if self.current_data is None or not self.memory: return None

        pdf_filename = "relatorio_eda.pdf"
        doc = SimpleDocTemplate(pdf_filename, pagesize=letter)
        styles = getSampleStyleSheet()
        story = []

        # Adicionar Título
        story.append(Paragraph("Relatório de Análise Exploratória de Dados (EDA)", styles['h1']))
        story.append(Spacer(1, 12))

        # Adicionar Conclusões
        conclusions = self.get_conclusions()
        if conclusions and conclusions != "Nenhum dado carregado." and not conclusions.startswith("Erro ao gerar conclusões:"):
            story.append(Paragraph("Conclusões e Insights:", styles['h2']))
            # Quebrar o texto das conclusões em parágrafos para ReportLab
            for line in conclusions.split('\n'):
                if line.strip(): # Ignorar linhas vazias
                    story.append(Paragraph(line, styles['Normal']))
                story.append(Spacer(1, 6)) # Espaço entre as linhas/parágrafos
            story.append(Spacer(1, 12))

        # Adicionar Gráficos (se disponíveis em bytes)
        # Mantendo a tentativa de adicionar imagens, mas lidando com a falha
        if self.generated_image_bytes and any(self.generated_image_bytes.values()): # Verifica se há *algum* byte de imagem gerado
            story.append(Paragraph("Gráficos Gerados:", styles['h2']))
            story.append(Spacer(1, 12))
            print(f"ℹ️ Tentando adicionar {len([b for b in self.generated_image_bytes.values() if b is not None])} imagem(ns) ao PDF...") # Depuração: contar apenas imagens com bytes
            for key, img_bytes in self.generated_image_bytes.items():
                if img_bytes: # Verificar se os bytes existem
                    try:
                        print(f"➡️ Adicionando imagem '{key}' ao PDF...") # Depuração
                        # Criar ImageReader a partir dos bytes
                        img_reader = ImageReader(img_bytes)
                        img = Image(img_reader)

                        # Redimensionar a imagem para caber na página, mantendo a proporção
                        img_width, img_height = img.drawWidth, img.drawHeight
                        page_width, page_height = letter
                        max_width = page_width - 2 * doc.leftMargin
                        max_height = page_height - 2 * doc.bottomMargin

                        aspect = img_height / float(img_width)
                        if img_width > max_width:
                            img_width = max_width
                            img_height = img_width * aspect
                        if img_height > max_height:
                             img_height = max_height
                             img_width = img_height / aspect

                        img.drawWidth = img_width
                        img.drawHeight = img_height

                        story.append(Paragraph(key.replace('_', ' ').title(), styles['h3']))
                        story.append(Spacer(1, 6))
                        story.append(img)
                        story.append(Spacer(1, 12)) # Espaço após o gráfico
                        print(f"✅ Imagem '{key}' adicionada com sucesso ao story.") # Depuração

                    except Exception as e:
                        print(f"❌ Erro ao adicionar imagem '{key}' ao PDF: {e}") # Depuração
                        story.append(Paragraph(f"Erro ao carregar ou adicionar imagem '{key}'.", styles['Normal']))
                        story.append(Spacer(1, 6))
                else:
                     print(f"⏭️ Pulando imagem '{key}': Bytes não foram gerados (erro na exportação).") # Depuração
                     # Adicionar uma nota no PDF para cada imagem que falhou
                     story.append(Paragraph(f"Imagem '{key}' não gerada (erro na exportação).", styles['Normal']))
                     story.append(Spacer(1, 6))

        elif self.current_data is not None: # Adicionar nota se não houver *nenhuma* imagem gerada mas dados existirem
             story.append(Paragraph("Gráficos Gerados:", styles['h2']))
             story.append(Spacer(1, 12))
             story.append(Paragraph("Não foi possível incluir os gráficos no relatório PDF devido a problemas técnicos na exportação de imagens.", styles['Normal']))
             story.append(Spacer(1, 12))


        try:
            doc.build(story)
            print(f"✅ Relatório PDF '{pdf_filename}' gerado com sucesso.")
            return pdf_filename
        except Exception as e:
            print(f"❌ Erro ao construir o PDF: {e}")
            return None

    def generate_html_report(self) -> Optional[str]:
        """Generates an HTML report with conclusions and embedded graphs."""
        if self.current_data is None or not self.memory:
            print("⚠️ Nenhum dado carregado ou nenhuma análise na memória para gerar relatório HTML.") # Depuração
            return None

        html_filename = os.path.join(temp_dir, "relatorio_eda.html")
        conclusions = self.get_conclusions()

        # Gerar HTML para os gráficos
        chart_html = ""
        print(f"ℹ️ Verificando figuras geradas para incluir no HTML: {len(self.generated_figures)}") # Depuração
        if self.generated_figures:

            for key, fig in self.generated_figures.items():
                try:
                    print(f"➡️ Convertendo gráfico '{key}' para HTML...") # Depuração
                    # Converter figura Plotly para HTML
                    fig_html = fig.to_html(full_html=False, include_plotlyjs='cdn') # Usar CDN para plotly.js
                    chart_html += f"""
                    <div class="graph">
                        <h3>{key.replace('_', ' ').title()}</h3>
                        {fig_html}
                    </div>
                    """
                    print(f"✅ Gráfico '{key}' convertido para HTML e incluído.") # Depuração
                except Exception as e:
                    print(f"❌ Erro ao converter gráfico '{key}' para HTML: {e}") # Depuração
                    chart_html += f"""
                    <div class="graph">
                        <h3>{key.replace('_', ' ').title()}</h3>
                        <p>Não foi possível renderizar o gráfico '{key}'.</p>
                    </div>
                    """
        else:
             print("⚠️ Nenhuma figura gerada para incluir no relatório HTML.") # Depuração
             chart_html = "<p>Nenhum gráfico gerado para incluir neste relatório.</p>"


        html_content = """
<!DOCTYPE html>
<html>
<head>
    <title>Relatório de Análise Exploratória de Dados (EDA)</title>
    <meta charset="utf-8"> <!-- Garantir codificação UTF-8 -->
    <style>
        body {{ font-family: sans-serif; margin: 20px; }}
        h1, h2, h3 {{ color: #333; }}
        .section {{ margin-bottom: 20px; padding-bottom: 10px; border-bottom: 1px solid #eee; }}
        .graph {{ margin-top: 15px; text-align: center; }}
        /* Remover regras de imagem estática */
        /* .graph img {{ max-width: 100%; height: auto; }} */
    </style>
</head>
<body>
    <h1>Relatório de Análise Exploratória de Dados (EDA)</h1>
    <div class="section">
        <h2>Conclusões e Insights</h2>
        {}
    </div>
    <div class="section">
        <h2>Gráficos Gerados</h2>
        {}
    </div>
</body>
</html>
""".format(
            # Format conclusions
            "".join(f"<p>{line}</p>" for line in conclusions.split('\n') if line.strip()) if conclusions and conclusions != "Nenhum dado carregado." and not conclusions.startswith("Erro ao gerar conclusões:") else "<p>Nenhuma conclusão gerada.</p>",
            # Format charts as HTML
            chart_html
        )

        try:
            with open(html_filename, "w", encoding="utf-8") as f:
                f.write(html_content)
            print(f"✅ Relatório HTML '{html_filename}' gerado com sucesso.") # Depuração
            return html_filename
        except Exception as e:
            print(f"❌ Erro ao gerar relatório HTML: {e}") # Depuração
            return None


def create_gradio_interface():
    """Cria e configura a interface Gradio para o agente EDA."""
    agent: Optional[CSVAnalysisAgent] = None
    latest_conclusions: str = ""
    # generated_images: List[str] = [] # Não precisamos mais da lista de caminhos de arquivo

    # Mover as funções handlers para o topo para garantir que estejam definidas antes de serem usadas
    def initialize_agent() -> str:
        nonlocal agent
        try:
            api_key = userdata.get('GEMINI_API_KEY')
            agent = CSVAnalysisAgent(api_key=api_key)
            return "✅ Agente inicializado com sucesso! Pronto para carregar dados."
        except Exception as e:
            return f"❌ Erro ao inicializar agente: {e}. Verifique a API Key."

    # Removido generated_images do retorno, pois não listamos mais arquivos
    def load_file(file: gr.File) -> str:
        nonlocal agent # generated_images
        if agent is None: return "❌ Inicialize o agente primeiro!" # , []
        if file is None: return "⚠️ Nenhum arquivo selecionado." # , []
        # generated_images.clear() # Não precisamos mais limpar a lista de caminhos de arquivo
        status = agent.load_csv_file(file.name)
        return status # , []

    # Simplificado o retorno para apenas texto e figura
    def update_images_list(analysis_text, fig): # Removido img_path
        # nonlocal generated_images # Não precisamos mais desta lista para exibição
        # if img_path and os.path.exists(img_path):
        #     generated_images.append(img_path)
        return analysis_text, fig

    # Funções handlers para os botões de análise que retornam texto e figura
    def get_data_types_interface():
        if agent is None: return "❌ Inicialize o agente primeiro!", None
        analysis_text, fig = agent.analyze_data_types()
        return update_images_list(analysis_text, fig)

    def detect_outliers_interface():
        if agent is None: return "❌ Inicialize o agente primeiro!", None
        analysis_text, fig = agent.detect_outliers()
        return update_images_list(analysis_text, fig)

    def correlation_analysis_interface():
        if agent is None: return "❌ Inicialize o agente primeiro!", None
        analysis_text, fig = agent.correlation_analysis()
        return update_images_list(analysis_text, fig)

    # Função handler para gerar e baixar o PDF
    def generate_and_download_pdf_interface() -> Optional[str]:
        nonlocal agent
        if agent is None: return None
        pdf_path = agent.generate_pdf_report()
        return pdf_path

    # Função handler para gerar e retornar o arquivo HTML de relatório completo
    def generate_and_download_html_report_interface() -> Optional[str]:
        nonlocal agent
        if agent is None: return None
        html_path = agent.generate_html_report()
        return html_path

    # Função handler para salvar gráficos individuais como HTML
    def save_plots_as_html_interface() -> gr.File | None: # Retorno pode ser None
        nonlocal agent
        if agent is None: return None
        saved_files = agent.save_plots_as_html()
        if saved_files:
            # Retorna o primeiro arquivo para download
            # Como o usuário relatou ver apenas 1 arquivo, talvez retornar todos como lista?
            # Gradio File component pode aceitar lista para download zipado.
            # Vamos tentar retornar a lista completa.
            return gr.File(value=saved_files, visible=True, label="Download Gráficos HTML Individuais")
        else:
            return None # Retorna None se nenhum arquivo foi salvo


    def get_conclusions_interface() -> str:
        nonlocal agent, latest_conclusions
        if agent is None: return "❌ Inicialize o agente primeiro!"
        latest_conclusions = agent.get_conclusions()
        return latest_conclusions

    # Removida a função de download de conclusões em TXT
    # def download_conclusions() -> Optional[str]:
    #     nonlocal latest_conclusions
    #     if not latest_conclusions: return None
    #     file_path = "conclusoes_eda.txt"
    #     with open(file_path, "w", encoding="utf-8") as f:
    #         f.write(latest_conclusions)
    #     return file_path

    # Esta função não é mais necessária ou funcional da mesma forma
    # Mantida como placeholder ou removida dependendo da necessidade futura de listar *algo*
    def list_generated_images_interface():
         # nonlocal generated_images # Não precisamos mais desta lista para exibição
         # return [gr.File(value=path, visible=True) for path in generated_images]
         # Poderia listar as chaves das imagens em memória, mas a galeria espera caminhos de arquivo ou URIs de imagem
         return [] # Retorna lista vazia já que não estamos salvando arquivos de imagem


    with gr.Blocks(title="Agente EDA", theme=gr.themes.Soft()) as interface:
        gr.Markdown("# 🤖 Agente Inteligente para Análise Exploratória de Dados (EDA)")
        with gr.Row():
            with gr.Column():
                init_btn = gr.Button("🚀 Inicializar Agente", variant="primary")
                init_output = gr.Textbox(label="Status da Inicialização", interactive=False)
                file_input = gr.File(label="📁 Upload do arquivo CSV", file_types=[".csv"])
                load_output = gr.Textbox(label="Status do Carregamento", interactive=False)

        with gr.Tabs():
            with gr.TabItem("📊 Análises Automáticas"):
                with gr.Row():
                    types_btn = gr.Button("🔍 Tipos de Dados")
                    stats_btn = gr.Button("📈 Resumo Estatístico")
                    outliers_btn = gr.Button("⚠️ Detectar Outliers")
                    corr_btn = gr.Button("🔗 Correlação")
                analysis_output = gr.Textbox(label="Resultado da Análise", lines=8, interactive=False)
                analysis_plot = gr.Plot(label="Visualização da Análise")

            with gr.TabItem("🎯 Conclusões e Download"):
                conclusions_btn = gr.Button("🧠 Gerar Conclusões Finais", variant="secondary")
                conclusions_output = gr.Textbox(label="Conclusões e Insights", lines=15, interactive=False)
                with gr.Row():
                    # Removido o botão e componente de download TXT
                    # download_txt_btn = gr.Button("💾 Baixar Conclusões (.txt)") # Renomeado para clareza
                    # download_txt_file_output = gr.File(label="Arquivo de Conclusões (.txt)", interactive=False) # Renomeado
                    pass # Usado para manter a estrutura da linha sem o componente removido

                gr.Markdown("### Relatório Completo (PDF):")
                with gr.Row():
                     generate_pdf_btn = gr.Button("📄 Gerar Relatório PDF", variant="secondary") # Novo botão
                     download_pdf_file_output = gr.File(label="Relatório EDA (.pdf)", interactive=False) # Novo componente de download de PDF

                gr.Markdown("### Relatório Completo (HTML):")
                with gr.Row():
                    generate_html_report_btn = gr.Button("📄 Gerar Relatório HTML", variant="secondary") # Novo botão
                    download_html_report_output = gr.File(label="Relatório EDA (.html)", interactive=False) # Novo componente de download

                gr.Markdown("### Salvar Gráficos Individuais (HTML):")
                with gr.Row():
                    save_html_btn = gr.Button("💾 Salvar Gráficos Individuais como HTML") # Novo botão para salvar HTML individual
                    # Alterado para tentar retornar a lista completa de arquivos para download zipado pelo Gradio
                    download_individual_html_output = gr.File(label="Download Gráficos HTML Individuais (Zip)", interactive=False, visible=False) # Componente para download individual


                gr.Markdown("### Imagens Geradas: (Exibidas na aba 'Análises Automáticas'. Download ou listagem de arquivos estáticos não disponível.)")
                # Ocultar ou remover componentes de listagem/download de imagens de arquivo
                # refresh_images_btn = gr.Button("🔄 Atualizar Lista de Imagens") # Ocultar ou remover
                # image_files_gallery = gr.Gallery(label="Imagens Geradas", show_label=False, columns=4, object_fit="contain", visible=False) # Ocultar ou remover

        # Event Handlers
        init_btn.click(fn=initialize_agent, outputs=init_output)
        # Removido image_files_gallery do output
        file_input.change(fn=load_file, inputs=file_input, outputs=[load_output])
        stats_btn.click(fn=lambda: agent.statistical_summary() if agent else "Inicialize o agente.", outputs=analysis_output)
        # Removido image_files_gallery do output
        types_btn.click(fn=get_data_types_interface, outputs=[analysis_output, analysis_plot])
        outliers_btn.click(fn=detect_outliers_interface, outputs=[analysis_output, analysis_plot])
        corr_btn.click(fn=correlation_analysis_interface, outputs=[analysis_output, analysis_plot])
        conclusions_btn.click(fn=get_conclusions_interface, outputs=conclusions_output)
        # Removido handler download TXT
        # download_txt_btn.click(fn=download_conclusions, outputs=download_txt_file_output)
        # Handler para gerar e baixar o PDF
        generate_pdf_btn.click(fn=generate_and_download_pdf_interface, outputs=download_pdf_file_output) # Usando a nova função handler
        # Handler para gerar e baixar o relatório HTML completo
        generate_html_report_btn.click(fn=generate_and_download_html_report_interface, outputs=download_html_report_output) # Usando a nova função handler
        # Handler para salvar gráficos individuais como HTML
        save_html_btn.click(fn=save_plots_as_html_interface, outputs=download_individual_html_output)

        # Removido handler e componentes de listagem/download de imagens de arquivo
        # refresh_images_btn.click(fn=fn=list_generated_images_interface, outputs=image_files_gallery) # Removido o handler e mantido a função vazia


    return interface

✅ Pacote kaleido instalado/atualizado dentro da CÉLULA 3.


### **4) INICIALIZAÇÃO DA INTERFACE GRADIO**

In [ ]:
# CÉLULA 4: INICIALIZAR E EXECUTAR A APLICAÇÃO
import asyncio
import platform

# Tentar resolver problemas de loop de evento em alguns ambientes
# Verifica se o sistema operacional é Windows antes de usar a política de loop padrão
# if platform.system() == 'Windows':
#     asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())

# Forçar um novo loop de evento para evitar conflitos (útil em Colab/Jupyter)
try:
    loop = asyncio.get_running_loop()
except RuntimeError: # No running loop
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)


print("🚀 Iniciando o Agente EDA...")
print("⏳ Aguarde a interface Gradio carregar e clique no link público abaixo para abrir o agente.")

try:
    interface = create_gradio_interface()
    interface.launch(
        share=True,        # Gera link público
        debug=True,        # Modo debug para ver logs de erro
        show_error=True    # Mostra erros na interface
    )
except Exception as e:
    print(f"❌ Falha ao iniciar a interface Gradio: {str(e)}")
    print("🔧 Verifique as mensagens de erro acima. Tente reiniciar o ambiente de execução.")

🚀 Iniciando o Agente EDA...
⏳ Aguarde a interface Gradio carregar e clique no link público abaixo para abrir o agente.
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7bb0e25bf69c5f27c1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


⚠️ Erro ao gerar bytes da imagem 'data_types_plot' (kaleido): 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

⚠️ Erro ao gerar bytes da imagem 'outliers_plot' (kaleido): 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido

✅ Relatório PDF 'relatorio_eda.pdf' gerado com sucesso.
ℹ️ Verificando figuras geradas para incluir no HTML: 2
➡️ Convertendo gráfico 'data_types_plot' para HTML...
✅ Gráfico 'data_types_plot' convertido para HTML e incluído.
➡️ Convertendo gráfico 'outliers_plot' para HTML...
✅ Gráfico 'outliers_plot' convertido para HTML e incluído.
✅ Relatório HTML 'temp_plots/relatorio_eda.html' gerado com sucesso.
ℹ️ Tentando salvar 2 figura(s) como HTML individual...
✅ Figura 'data_types_plot' salva como HTML em temp_plots/data_types_plot.html
✅ Figura 'outliers_plot' salva como HTML em temp_plots/outliers_plot.

---
### 📚 Recursos Adicionais


In [ ]:
# CÉLULA 5 (Opcional): LISTAR MODELOS DISPONÍVEIS
import google.generativeai as genai
from google.colab import userdata

try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)
    print("Listando modelos disponíveis que suportam 'generateContent':")
    for m in genai.list_models():
        if 'generateContent' in m.supported_generation_methods:
            print(m.name)
except Exception as e:
    print(f"Erro ao listar modelos: {e}")

Listando modelos disponíveis que suportam 'generateContent':
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-05-20
models/gemini-2.5-flash
models/gemini-2.5-flash-lite-preview-06-17
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-

* * *
### 📚 Adicionais e Diagnósticos

*   **Repositório GitHub do Agente EDA Inteligente:** Acesse o código-fonte e mais informações sobre o agente em https://github.com/MarisaDeM/EDA_INTELIGENTE